In [1]:
import pandas as pd
import metadata_parser as mp
import requests
from bs4 import BeautifulSoup as bs
import re
import undetected_chromedriver as uc
import time

# Login Bibliopass

Bibliopass is a remote publisher library owned by the University of Turin. Each remote publisher library may have a different login interface/scheme so you need to modify this login code. If you want to use a Scopus subscription account, you also need to modify this login code. If you do not have access to log in to Scopus, you cannot use this code.

In [2]:
scopus_url = "https://www-scopus-com.bibliopass.unito.it/"

In [3]:
driver = uc.Chrome()

In [4]:
driver.get(scopus_url)
time.sleep(5)

In [ ]:
try:
    username = driver.find_element_by_id("username")
    password = driver.find_element_by_id("password")
    username.send_keys("your_username")
    password.send_keys("your_password")
    driver.find_element_by_name("_eventId_proceed").click()
except:
    pass

# Get All Metadata

In [81]:
# Call the dataset
scopus = pd.read_excel("PaperLinks_Scopus.xlsx")

In [ ]:
# Login Bibliopass
scopus_url = "https://www-scopus-com.bibliopass.unito.it/"
driver = uc.Chrome()
driver.get(scopus_url)
time.sleep(5)
try: # login
    username = driver.find_element_by_id("username")
    password = driver.find_element_by_id("password")
    username.send_keys("your_username")
    password.send_keys("your_password")
    driver.find_element_by_name("_eventId_proceed").click()
except:
    pass # pass the login process if already login

list_year, list_title, list_conf_jour, list_authors, list_keywords, list_abstract = [], [], [], [], [], []

for paper_url in scopus["Paper Links"]:
    # Get Page
    driver.get(paper_url)
    time.sleep(5)
    
    # Get Year
    try:
        year_html = driver.find_element_by_css_selector("div[id='publicationYear']")
        year = year_html.get_attribute('innerHTML')
    except:
        year = "failed_to_scrap"
        
    # Get Title
    try:
        title_html = driver.find_element_by_css_selector("span[class='Highlight-module__1p2SO']")
        title = title_html.text
    except:
        title = "failed_to_scrap"
        
    # Get Conf/Journal Name
    try:
        conf_jour_html = driver.find_element_by_css_selector("div[id='publicationTitle']")
        conf_jour_name = conf_jour_html.get_attribute('innerHTML')
    except:
        conf_jour_name = "failed_to_scrap"
    
    # Get Authors
    try:
        get_authors_html = driver.find_element_by_css_selector("div[data-testid='author-list']")
        get_authors_string_dirty = str(get_authors_html.get_attribute('innerHTML'))
        get_authors_list_dirty = get_authors_string_dirty.split('screenreadertext="Send mail to ')[1:]
        authors_list = []
        for i in get_authors_list_dirty:
            authors_list.append(i.split('"')[0])    
        authors = ', '.join(authors_list)
    except:
        authors = "failed_to_scrap" 
    
    # Get Keywords
    try:
        get_keywords_html = driver.find_element_by_css_selector("div[class='margin-size-4-t']")
        get_keywords_string_dirty = str(get_keywords_html.get_attribute('innerHTML'))
        get_keywords_list_dirty = get_keywords_string_dirty.split('<span class="Highlight-module__1p2SO">')[1:]
        keywords_list = []
        for i in get_keywords_list_dirty:
            keywords_list.append(i.split('</span>')[0])    
        keywords = ', '.join(keywords_list)
    except:
        keywords = "failed_to_scrap"
        
    # Get Abstract
    try:
        get_abstract_html = driver.find_element_by_css_selector("div[class='margin-size-4-t margin-size-16-b']")
        get_abstract_string_dirty = str(get_abstract_html.get_attribute('innerHTML'))
        get_abstract_list_dirty = get_abstract_string_dirty.split('<span class="Highlight-module__1p2SO">')[1:]
        abstract = get_abstract_list_dirty[0].split(' ©')[0]
    except:
        abstract = "failed_to_scrap"
    
    # Append to the list
    list_year.append(year)
    list_title.append(title)
    list_conf_jour.append(conf_jour_name)
    list_authors.append(authors)
    list_keywords.append(keywords)
    list_abstract.append(abstract)

driver.close()

In [88]:
# Concatenate list as new column
scopus["Year"], scopus["Title"], scopus["Conf/Journal Name"] = list_year, list_title, list_conf_jour
scopus["Authors"], scopus["Keywords"], scopus["Abstract"] = list_authors, list_keywords, list_abstract

In [90]:
scopus.to_excel("PaperMetadata_Unclean_Scopus.xlsx", index = False)